In [46]:
# Standard imports
from os.path import join, isdir, isfile
from os import listdir as ls
import copy
import os
from IPython.display import display
from ipywidgets import interact, widgets
from PIL import Image
import numpy as np
import sys
import time
import shutil
import os
import matplotlib.pyplot as plt
import pandas
import networkx as nx

In [ ]:
# Local clone
! git clone https://github.com/nanopiero/annotation.git
import sys
sys.path.append('annotation')


In [10]:
# Import the functions we need to sort a sequence :
import importlib
import src.utils as utils
import src.poset_mergesort_functions_per_level  as pmf

In [ ]:
# Prepare a sequence to sort :

In [11]:
! pip install huggingface_hub
from huggingface_hub import hf_hub_download

In [ ]:
import os
import zipfile
from huggingface_hub import hf_hub_download

# Step 1: Download the zip file using hf_hub_download
local_zip = hf_hub_download(repo_id="nanopiero/weow", filename="webcam_images.zip")

# Step 2: Unzip the contents
with zipfile.ZipFile(local_zip, 'r') as zip_ref:
    zip_ref.extractall('webcam_images')

In [ ]:
# List all .jpg images in the directory
image_dir = 'webcam_images'
image_files = sorted([f for f in os.listdir(image_dir) if f.endswith('.jpg')], key=lambda x: int(x.split('.')[0]))

def show_image(index):
    img_path = os.path.join('webcam_images', image_files[index])
    img = Image.open(img_path)
    plt.imshow(img)
    plt.axis('off')  # Hide axes
    plt.show()

# Use a slider to scroll through the images
interact(show_image, index=widgets.IntSlider(min=0, max=len(image_files)-1, step=1, description="Image Index"));


In [59]:
# Prepare the folder for the sequence "0000":
! mkdir -p dataset/0000/labels_ord dataset/0000/images
! cp webcam_images/* dataset/0000/images/
# ! ls dataset/0000/images

In [60]:
# Get the dictionnary of snow_cover levels
# resulting from the image wise annotation step:
# (here, the dict is arbitrarily set to the level '0' for all images)
lbls = {n: {'sequence':'0000', 'levelsc':'0'} for n in ls('dataset/0000/images')}

In [61]:
# Config:
critere = 'sc'
param = 'sc'
mode = 'surface'
modes = [mode]
subgroup = ''
times = ['']

grounds = [r'snow_ground', r'snow_road', r'white_road', r'snow_ground_dry_road']

# path to the folder that contains sequences
dir_splitted = 'dataset'

# path to a tmp dir where sequences are sorted:
! mkdir -p tmp/images

In [ ]:
# Define the list of sequences to label and the dict seq2images
sequences = sorted({lbls[name]['sequence'] for name in lbls})

# Create a dictionary mapping sequences to image sets
seq2images = {
    sequence: {name for name in lbls if lbls[name]['sequence'] == sequence}
    for sequence in sequences
}

# List sequences that have more than one associated image
sequences_to_sort = [seq for seq, images in seq2images.items() if len(images) > 1]

print(sequences_to_sort, len(sequences_to_sort), len(sequences))


In [ ]:
# Generate new graphs dg, ug, eg:
graphs = utils.get_new_graphs(lbls)
tdg, tug, teg = utils.get_sdg_from_levels(graphs, lbls, param) # no supplementary edges

In [ ]:
# Add previous comparisons (stored in labels_ord) & Cleaning : one must not have cycles

graphs= (tdg, tug, teg)
tdg, tug, teg = utils.fill_graph_from_splitted(graphs, dir_splitted, ext='sc')
print(len(tdg.edges), len(tug.edges), len(teg.edges))


#washing
two_cycles = set(tdg.edges).intersection(utils.invert_edges(set(tdg.edges)))
two_cycles = utils.edges_and_inverted_edges(two_cycles)
print(two_cycles)

print(str(len(two_cycles)) + ' 2-cycles')
tdg.remove_edges_from(two_cycles)
tdg = nx.transitive_closure(tdg)
two_cycles = set(tdg.edges).intersection(utils.invert_edges(set(tdg.edges)))
two_cycles = utils.edges_and_inverted_edges(two_cycles)
print(str(len(two_cycles)) + ' 2-cycles after washing')


vs03 = utils.edges_and_inverted_edges(tug.edges).intersection(set(tdg.edges))
print(str(len(vs03)) + ' 0vs3')
vs03eg = {edge for edge in vs03 if edge in teg.edges}
tdg.remove_edges_from(vs03)
tug.remove_edges_from(vs03)
teg.remove_edges_from(vs03eg)

tdg = nx.transitive_closure(tdg)

vs03 = utils.edges_and_inverted_edges(tug.edges).intersection(set(tdg.edges))
print(str(len(vs03)) + ' 0vs3 after washing')
print(len(tdg.edges), len(tug.edges), len(teg.edges))

In [ ]:
! cd annotation ; git pull ; cd ..
# Reload each module if needed
importlib.reload(utils)
importlib.reload(pmf)

In [ ]:
# To get a decomposition in chains (do it with spyder/pycharm to work in interactive pop up window)

for i, sequence in enumerate(sequences_to_sort):
    print('sequence: ' + str(sequence), i)
    root_dataset = dir_splitted
    dataset = os.path.join(root_dataset, str(sequence))
    images_dataset = join(dataset, 'images')
    root_cs = join(dataset, r'labels_ord')

    if not isdir(root_cs):
        os.mkdir(root_cs)

    # init frame_lbls and dic of subgroup
    frame_lbls = {}
    for name in os.listdir(images_dataset):
            frame_lbls[name] = lbls[name]

    dic_of_subgroups = {}
    dic_of_subgroups[''] = [name for name in frame_lbls]

    labeling_root= "tmp"
    temp_images_dir= "tmp/images"
    name_poset_sc = r'poset_sc.pickle'

    #%def graph paths
    graphs_names = [os.path.join(graph + '_sc' +r'.gpickle') for graph in ["dg","ug","eg"]]
    graphs_paths = [os.path.join(root_cs, graph_name) for graph_name in graphs_names]

    #def nodes
    nodes = sorted(dic_of_subgroups[subgroup])
    print('labelling of ' + str(len(nodes)) + ' nodes')

    #clean images_dir
    for name in os.listdir(temp_images_dir):
        os.remove(os.path.join(temp_images_dir,name))

    #copy in the temp image dir
    for node in nodes:
        shutil.copy(os.path.join(dataset, 'images', node), os.path.join(temp_images_dir))

    # if graphs_names[0] not in os.listdir(root_cs):
    print("need to init graphs")
    # restrict tdg, tug, teg:
    dg = tdg.subgraph(nodes).copy()
    ug = tug.subgraph(nodes).copy()
    eg = teg.subgraph(nodes).copy()

    #save the graphs
    utils.write_gpickle(dg, graphs_paths[0])
    utils.write_gpickle(ug, graphs_paths[1])
    utils.write_gpickle(eg, graphs_paths[2])

    # else:
    #     pass


    kwargs = {'root_cs': root_cs,
              'critere':  'sc',
              'subgroup': subgroup,
              'mode' : '',
              'graphs_paths': graphs_paths,
              'images_dir' : temp_images_dir
              }

    decomposition = pmf.labelling_mode(lbls, **kwargs)

In [ ]:
# To remove / clean nodes :
#00010268_0_20131114_081603.jpg
ntr = ['00010657_7002_20130721_180838.jpg']

clean_nodes(ntr, **kwargs)

#%%
path2 = os.path.join(root_cs, 'decomposition_vvday.pickle')
os.remove(path2)